In [ ]:
# | default_exp dataset

# Dataset

> Protein Dataset

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from torch.utils.data import Dataset

/Users/education/DATA/projects/bio/progen/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ProteinDataset(Dataset):
    def __init__(self, tokenizer, path):
        pass

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return 1